# LangChain 核心模块 Agent 

有些应用程序不仅需要预先确定的LLM/其他工具调用链，而且可能需要根据用户输入的不同而产生不同的链条。

在这些类型的链条中，有一个“代理人”可以访问一套工具。

根据用户输入，代理人可以决定是否调用其中任何一个工具。

在本文档的本节中，我们首先从“快速入门”开始，介绍如何以端到端的方式使用与代理人相关的所有内容。

## 1.OPENAI Function

In [2]:
import os
import socket
import socks

socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 10808)
socket.socket = socks.socksocket

In [3]:
from langchain.chat_models import ChatOpenAI

# 使用 GPT-3.5-turbo
llm = ChatOpenAI(temperature=0)

d:\sdk\Anaconda3\envs\bigmodel\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:

from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [5]:
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent

system_message = SystemMessage(content="你是非常强大的AI助手，但在计算单词长度方面不擅长。")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

In [6]:
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)


d:\sdk\Anaconda3\envs\bigmodel\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.openai_functions_agent.base.OpenAIFunctionsAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_openai_functions_agent instead.
  warn_deprecated(


In [7]:
from langchain.agents import AgentExecutor

# 实例化 OpenAIFunctionsAgent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.run("单词“educa”中有多少个字母?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5单词“educa”中有5个字母。

> Finished chain.


'单词“educa”中有5个字母。'

## 2.ReAct
ReAct 核心思想是 推理+操作，本示例以 Google Search 和 LLM Math 作为可选操作集合（toolkits），实现 ReAct 功能。

In [2]:
import os
import socket
import socks

socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 10808)
socket.socket = socks.socksocket

#os.environ['SERPAPI_API_KEY'] = ''


In [3]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)


d:\sdk\Anaconda3\envs\bigmodel\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [4]:

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

#加载 LangChain 内置的 Tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [5]:
# 实例化 ZERO_SHOT_REACT Agent
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 运行 Agent    
# agent.run("谁是莱昂纳多·迪卡普里奥的女朋友？她现在年龄的0.43次方是多少?")


d:\sdk\Anaconda3\envs\bigmodel\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [15]:
agent.run("促肾上腺皮质激素偏低，结果是14.9pg/ml.标准值是25-100pg/ml，这个检查结果严重吗")



> Entering new AgentExecutor chain...
 I should search for information about adrenal cortex hormones and their normal levels
Action: Search
Action Input: "adrenal cortex hormones normal levels"
Observation: The normal level of adrenal gland hormone
Thought: I should use a calculator to determine the percentage of the given result compared to the normal range
Action: Calculator
Action Input: (14.9/25)*100
Observation: Answer: 59.599999999999994
Thought: I now know that the given result is significantly lower than the normal range
Final Answer: Yes, the given result of 14.9pg/ml is significantly lower than the normal range of 25-100pg/ml, indicating a potential issue with adrenal gland function.

> Finished chain.


'Yes, the given result of 14.9pg/ml is significantly lower than the normal range of 25-100pg/ml, indicating a potential issue with adrenal gland function.'

## 3.Self ask with search

In [19]:

from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0)

In [20]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [21]:

# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [22]:

# 实际运行 Agent，查询问题（正确）
self_ask_with_search.run(
    "What is the hometown of the reigning men's U.S. Open champion?"
)



> Entering new AgentExecutor chain...


 Yes.
Follow up: Who is the reigning men's U.S. Open champion?
Intermediate answer: Novak Djokovic
Follow up: Where is Novak Djokovic from?
Intermediate answer: Belgrade, Serbia
So the final answer is: Belgrade, Serbia

> Finished chain.


'Belgrade, Serbia'

In [25]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.run(
    "How old is the wife of President Xijinping?"
)



> Entering new AgentExecutor chain...


 Yes.
Follow up: Who is President Xijinping's wife?
Intermediate answer: ["On 1 September 1987, she married Xi Jinping, then the Vice Mayor of Xiamen. In May 1990, she defended her thesis and received her master's degree, becoming the ...", "Who is Xi Jinping's wife? Meet Peng Liyuan, the famous folk singer who helped pave the Chinese president's path to power. By Kate Wong.", 'While living in Fujian, Xi married the well-known folk singer Peng Liyuan in 1987. He continued to work his way upward, and by 1995 he had ascended to the post ...', "Communist Party's leader Xi Jinping's wife, famed military singer Peng Liyuan, found herself in the news again as a model Party member more than ready to answer ...", "She is the wife of Xi Jinping, the current general secretary of the Chinese Communist Party and president of the People's Republic of China. Peng Liyuan.", 'On the evening of November 5, 2019, President Xi Jinping and his wife Peng Liyuan met with President Emmanuel Macron of France 

'61 years old'